In [1]:
import os

try:
    GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
    if(GOOGLE_API_KEY):
        print("Key found")
    else:
        print("Key not found")
except Exception as e:
    print(f"Error:{e}")

Key found


In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService, InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("ADK components imported ")


ADK components imported 


In [4]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# 1.1 Stateful Agent

In [6]:
APP_NAME = "default"
USER_ID = "default"
SESSION = "default"

MODEL_NAME = "gemini-2.5-flash-lite"

root_agent = LlmAgent(
    name = "text_chat_bot",
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    description= "A text chatbot"
)

session_service = InMemorySessionService()

runner = Runner(agent = root_agent, app_name = APP_NAME, session_service = session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

App name mismatch detected. The runner is configured with app name "default", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


In [9]:
await run_session(
    runner,
    [
        "Hi, I am Sam! What is the capital of United States?",
        "Hello! What is my name?",  # This time, the agent should remember!
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hi, I am Sam! What is the capital of United States?
gemini-2.5-flash-lite >  The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  You told me your name is Sam!


In [11]:
# Run this cell after restarting the kernel. All this history will be gone...
await run_session(
    runner,
    ["What did I ask you about earlier?", "And remind me, what's my name?"],
    "state-agentic-session",
)  # Note, we are using same session name


 ### Session: state-agentic-session

User > What did I ask you about earlier?
gemini-2.5-flash-lite >  I am sorry, but I do not have access to your past conversations. I can only process the current conversation. Therefore, I cannot tell you what you asked me about earlier.

User > And remind me, what's my name?
gemini-2.5-flash-lite >  I am sorry, but I do not have access to your personal information, including your name. I am a text chatbot and do not store any user data.


# 1.2 Persistent Sessions

In [14]:
chatbot_agent = LlmAgent(
    model = Gemini(
        model = "gemini-2.5-flash-lite",
        retry_options = retry_config
    ),
    name = "text_chat_bot",
    description = "A text chatbot with persistent memory"
)

db_url = "sqlite:///my_agent_data.db"
session_service = DatabaseSessionService(db_url)

runner = Runner(agent = chatbot_agent, app_name = APP_NAME, session_service = session_service)
print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

App name mismatch detected. The runner is configured with app name "default", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [15]:
await run_session(
    runner,
    ["Hi, I am Sam! What is the capital of the United States?", "Hello! What is my name?"],
    "test-db-session-01",
)


 ### Session: test-db-session-01

User > Hi, I am Sam! What is the capital of the United States?
gemini-2.5-flash-lite >  Hi Sam! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite >  Your name is Sam.


In [16]:
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)


check_data_in_db()

['app_name', 'session_id', 'author', 'content']
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hi, I am Sam! What is the capital of the United States?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Hi Sam! The capital of the United States is Washington, D.C."}], "role": "model"}')
('default', 'test-db-session-01', 'user', '{"parts": [{"text": "Hello! What is my name?"}], "role": "user"}')
('default', 'test-db-session-01', 'text_chat_bot', '{"parts": [{"text": "Your name is Sam."}], "role": "model"}')


# 1.3 Context Compaction

In [17]:
# Re-define our app with Events Compaction enabled
research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅ Research App upgraded with Events Compaction!")

C:\Users\aarya\AppData\Local\Temp\ipykernel_2592\3773147741.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(
App name mismatch detected. The runner is configured with app name "research_app_compacting", but the root agent was loaded from "C:\Users\aarya\Desktop\Aaryan\Projects\Agents\ai-agents-workshop\agentenv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Research App upgraded with Events Compaction!


In [18]:
await run_session(
    research_runner_compacting,
    "What is the latest news about AI in healthcare?",
    "compaction_demo",
)


 ### Session: compaction_demo

User > What is the latest news about AI in healthcare?
gemini-2.5-flash-lite >  Here's a summary of the latest news and trends in AI in healthcare, covering a few key areas:

**1. Advancements in Diagnostics and Imaging:**

*   **AI for Early Disease Detection:** There's a lot of ongoing research and development in using AI to analyze medical images (X-rays, CT scans, MRIs, etc.) for earlier and more accurate detection of diseases like cancer, diabetic retinopathy, and Alzheimer's. Companies are developing AI algorithms that can identify subtle patterns that human radiologists might miss.
*   **Personalized Treatment Recommendations:** AI is being used to analyze vast amounts of patient data, including genetic information, medical history, and treatment outcomes, to suggest the most effective and personalized treatment plans. This is particularly promising in areas like oncology.
*   **Drug Discovery and Development Acceleration:** AI is revolutionizing 

In [19]:
await run_session(
    research_runner_compacting,
    "Are there any new developments in drug discovery?",
    "compaction_demo",
)


 ### Session: compaction_demo

User > Are there any new developments in drug discovery?
gemini-2.5-flash-lite >  Yes, absolutely! Drug discovery is one of the areas where AI is having the most profound and rapid impact. Here are some of the key new developments and trends:

**1. Generative AI and De Novo Drug Design:**

*   **Designing Novel Molecules from Scratch:** Generative AI models (like GANs and diffusion models) are no longer just screening existing compounds. They can now design entirely *new* molecular structures with desired properties, optimized for binding to a specific target, and even with improved pharmacokinetic profiles. This is a massive leap from traditional methods.
*   **Targeted Drug Design:** Instead of a broad search, generative AI can be used to design molecules that are highly specific to a particular protein or biological pathway involved in a disease, potentially leading to more effective and less toxic drugs.

**2. AI-Accelerated Target Identification:**


In [20]:
await run_session(
    research_runner_compacting,
    "Tell me more about the second development you found.",
    "compaction_demo",
)


 ### Session: compaction_demo

User > Tell me more about the second development you found.
gemini-2.5-flash-lite >  You're referring to **AI for Target Identification** in drug discovery. This is a really crucial area because finding the right biological target is the first step in developing any drug. If you aim for the wrong target, the drug won't work, no matter how well-designed it is.

Here's a deeper dive into how AI is revolutionizing target identification:

**The Traditional Challenge:**

Historically, identifying drug targets relied heavily on:

*   **Hypothesis-driven research:** Scientists would form hypotheses based on existing biological knowledge and then conduct experiments to test them. This is often slow, labor-intensive, and can be limited by human biases or incomplete understanding.
*   **Observational studies:** Observing disease patterns and trying to infer underlying biological mechanisms.

This process could take years, and many promising avenues might have been

In [22]:
final_session = await session_service.get_session(
    app_name=research_runner_compacting.app_name,
    user_id=USER_ID,
    session_id="compaction_demo",
)

print("--- Searching for Compaction Summary Event ---\n")
found_summary = False
for event in final_session.events:
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print("\n✅ SUCCESS! Found the Compaction Event:\n")
        print(f"  Author: {event.author}\n")
        print(f"\n Compacted information: {event}\n")
        found_summary = True
        break

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---


✅ SUCCESS! Found the Compaction Event:

  Author: user


 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None invocation_id='1ea0d247-cb6a-4b83-92c5-b02e390472fd' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction={'start_timestamp': 1763105881.679013, 'end_timestamp': 1763105915.924103, 'compacted_content': {'parts': [{'function_call': None, 'code_execution_result': None, 'executable_code': None, 'file_data': None, 'function_response': None, 'inline_da

In [23]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'my_agent_data.db'